Submission

In [ ]:
import torch
from torchvision import transforms, datasets
import numpy as np
from torch.utils.data.sampler import *
import torch.nn as nn

from pprint import pformat

torch.multiprocessing.set_sharing_strategy('file_system')


class LogisticRegression(nn.Module):
    def __init__(self, params):
        super(LogisticRegression, self).__init__()
        # your code here 
        #*** extract input dimension and output dimension from params ***#
        dim = params['dimension']
        output = params['output']
        self.rf = nn.Linear(dim,output) # initialize model


    def forward(self, x):
        out = None
        # your code here
        x = x.view(x.size(0),-1)
        out = self.rf(x)
        return out


def get_dataset(dataset_name):

    # Ref: copied from MNIST_Multiple_Linear_Regression.ipynb on eclass to load data set
    #*** get both trainning and testing data from torch vision ***#
    if dataset_name == "MNIST":
      
      # Ref: copied from MNIST_Multiple_Linear_Regression.ipynb on eclass to load data set
      training = datasets.MNIST('/MNIST_dataset/', train=True, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))]))
      # Ref: copied from MNIST_Multiple_Linear_Regression.ipynb on eclass to load data set
      test_set = datasets.MNIST('/MNIST_dataset/', train=False, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))]))
      
      # spliting training_set and valid_set with specific indices
      indices = np.arange(0,60000)
      training_set = torch.utils.data.Subset(training, indices[:48000])
      validation_set = torch.utils.data.Subset(training, indices[48000:])


    elif dataset_name == "CIFAR10":

      # Ref: copied from CIFAR10_Multiple_Linear_Regression.ipynb on eclass to load data set
      transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
      CIFAR10_training = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
      test_set = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

      indices = np.arange(0,50000)
      training_set = torch.utils.data.Subset(CIFAR10_training, indices[:38000])
      validation_set = torch.utils.data.Subset(CIFAR10_training, indices[38000:])
    else:
      raise Exception(dataset_name)

    # set up batch_size for train and test
    batch_size_train = 128
    batch_size_test = 1000

    train_dataloader = torch.utils.data.DataLoader(training_set,batch_size=batch_size_train, shuffle=True)
    valid_dataloader = torch.utils.data.DataLoader(validation_set,batch_size=batch_size_train, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test_set,batch_size=batch_size_test, shuffle=True)

    

    mnist_params = {  # set parameters mnist
        'dimension':784,  # dimension of each input image 784 x 1
        'train_size':48000,     # total size of trainning set
        'output':10,      # output size => 10 classes
        'type':'mnist',     # type indicates MNIST or CIFAR10
        'epochs':10,      # epochs for this data set
        'learning_rate':3e-3,     # learning rate for the model
        'opt':'SGD',  # opt: which optimizer is used in this dataset
        'wd':1e-3,  # wd: weight value as Regularization L2
        'mo':0.9  # mo: Momentum value if using SGD optimizer
    }

    # set parameters cifar10
    cifar10_params = {
        # your code here
        'dimension':3072, # dimension of each input image 784 x 1
        'train_size':38000,  # total size of trainning set
        'output':10,  # output size => 10 classes
        'type':'cifar', # type indicates MNIST or CIFAR10
        'epochs':10, # epochs for this data set
        'learning_rate':1e-3, # learning rate for the model
        'opt':'SGD',  # opt: which optimizer is used in this dataset
        'wd':1e-3,  # wd: weight value as Regularization L2
        'mo':0  # mo: Momentum value if using SGD optimizer
    }

    # ******** There is nothing else under this if statement as I have already handle  ****#
    # ******** those two set of images at above if statement. So here is only matching ****#
    # ******** the parameters as default                             ****#
    if dataset_name == "MNIST":
        params = mnist_params

        # your code here  ***cross out***
    elif dataset_name == "CIFAR10":
        params = cifar10_params
        # your code here  ***cross out***
    else:
        raise AssertionError(f'Invalid dataset: {dataset_name}')

    dataloaders = {
        'train': train_dataloader,
        'valid': valid_dataloader,
        'test': test_dataloader,
    }
    return dataloaders, params
###########################################################################################################

def test(model, test_dataloader, device, params):
    test_predictions = []
    true_labels = []

    # your code here
    model.eval()  # set model to evaluation

    # ****** This part below is adopted from lecture notebook, which calls our model ****#
    # ****** for output, appends them with true lables the to given lists       ****#
    with torch.no_grad():
      for data,target in test_dataloader:
        data = data.to(device)
        output = model(data)
        a = output.data.max(1,keepdim=False)[1]
        test_predictions.append(a)
        true_labels.append(target) 

    return test_predictions, true_labels


def validate(model, valid_dataloader, device, params):
    mean_acc = None

    # your code here
    model.eval()  # set model to eval mode
    correct = 0 # used for mean_acc calculation

    # ****** The following part is adopted from lecture notebook, multilinear regression.   ***#
    # ****** It is similar to test method whereas validation uses part of the training   ***#
    # ****** set to test the model. mean_acc is calculated by total all correct number   ***#
    # ****** over size of validation set                              ***#

    with torch.no_grad():
      for data,target in valid_dataloader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        pred = output.data.max(1,keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()

    correct = correct.cpu().numpy()
    mean_acc = correct/12500
    return mean_acc


def train(model, train_dataloader, valid_dataloader, device, params):
    mean_train_loss = 0

    # your code here

    # ***** extract essential value from params, learning rate, weight decay,momentum ***#
    lr_rate = params['learning_rate']
    wd = params['wd']
    mo = params['mo']

    ## initialize optimizer, either SGD or Adam
    if params['opt'] == 'SGD':
      optimizer = torch.optim.SGD(model.parameters(),lr = lr_rate,  = 1e-3,momentum=mo)
    elif params['opt'] == 'Adam':
      optimizer = torch.optim.Adam(model.parameters(),lr = lr_rate)

    epochs = params['epochs'] # define epochs value for certain dataset

    # ****** The code below is similar to lecture notebook as it is adopted from them ***#
    # ****** and the difference is that cross entropy loss function being used here as ***#
    # ****** logistic regression model. Basiclly it gets outputs from model and target  ***# 
    # ****** from dataset and compare with loss function, then use back prop to make  ***#
    # ****** model learn from each batch                            ***# 
    for epoch in range(epochs):
      model.train()
      for batch_idx,(data,target) in enumerate(train_dataloader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.functional.cross_entropy(output,target)
        mean_train_loss += loss
        loss.backward()
        optimizer.step()

      model.eval()
      if epoch%5==0:  # After each 5 epochs, call validate to prevent over fitting
        validate(model,valid_dataloader,device,params)
    
    mean_train_loss = mean_train_loss.detach().cpu().numpy() / params['train_size']

    return mean_train_loss


def tune_hyper_parameter(dataloaders, device, params):
    """update the following in your search"""
    best_optimizer = "Adam"
    best_hyperparams = {
        "regularizer": {
            # your code here
            'value':0
        },
        "Adam": {
            "accuracy": 0,
            "learning_rate": 0,
        },
        "SGD": {
            "accuracy": 0,
            "learning_rate": 0,
            "momentum": 0,
        }
    }
    """accumulate all validation accuracies you compute during hyper parameter search 
    for both optimizers"""
    validation_accuracy = []

    # your code here
    lr_set = [1e-2,1e-3,] # my learning rate set
    momentum_set = [0,0.9]  # my momentum set
    weight_decay_set = [0,1e-3] # my weight_decay_set / regularizer
    mean_loss=[100,100] # store mean_loss value

    # ****** Grid Search is used here to go through all possible combinations of hyperparameters ***#
    for lr in lr_set:
      for mo in momentum_set:
        for wd in weight_decay_set:
          # ******* For each combination, SGD optimizer is tested, then Adam ***#
          params['lr_rate'] = lr
          params['mo'] = mo
          params['wd'] = wd
          # *** SGD traning *** #
          params['opt'] = 'SGD'
          model = LogisticRegression(params).to(device) # set up trainning model
          c_loss = train(model,dataloaders['train'],dataloaders['valid'],device,params)  # run trainning

          best_acc = best_hyperparams['SGD']['accuracy']
          acc = validate(model,dataloaders['valid'],device,params)  # check accuracy on validation
          
          if acc>best_acc and mean_loss[0] > c_loss:  # update best combination if accuacy improved
            best_hyperparams['SGD']['accuracy'] = acc
            best_hyperparams['SGD']['learning_rate'] = lr
            best_hyperparams['SGD']['momentum'] = mo
            mean_loss[0] = c_loss
            validation_accuracy.append(acc)

          # ****** Now test Adam optimizer, same algorithm as above ***#
          # Adam training
          params['opt'] = 'Adam'
          model = LogisticRegression(params).to(device) 
          c_lose = train(model,dataloaders['train'],dataloaders['valid'],device,params)
          best_acc = best_hyperparams['Adam']['accuracy']
          acc = validate(model,dataloaders['valid'],device,params)
          if acc>best_acc and mean_loss[1] > c_loss:
            best_hyperparams['Adam']['accuracy'] = acc
            best_hyperparams['Adam']['learning_rate'] = lr
            mean_loss[1] - c_loss
            validation_accuracy.append(acc)
    
    # check which optimizer combination gives best accuracy
    if best_hyperparams['Adam']['accuracy'] < best_hyperparams['SGD']['accuracy']:
      best_optimizer = 'SGD'




    print("\nOptimal performance: Validation Accuracy: {:.3f}, "
          "with {:s} optimizer "
          "using hyper parameters:\n{:s} ".format(
        max(validation_accuracy),
        best_optimizer,
        pformat(best_hyperparams[best_optimizer])))

    print("\nOptimal regularization hyper parameters:\n{:s} ".format(
        pformat(best_hyperparams['regularizer'])))


Part1

In [ ]:
#@title
import torch
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat

#from A2_submission import LogisticRegression, get_dataset, train, test


torch.multiprocessing.set_sharing_strategy('file_system')


def compute_score(acc, run_time, min_thres, max_thres, max_run_time):
    if run_time > max_run_time:
        return 0.0
    print(run_time)
    if acc <= min_thres:
        base_score = 0.0
    elif acc >= max_thres:
        base_score = 100.0
    else:
        base_score = float(acc - min_thres) / (max_thres - min_thres) \
                     * 100
    return base_score


def run(dataset_name, device):
    dataloaders, params = get_dataset(dataset_name)

    start = timeit.default_timer()

    model = LogisticRegression(params).to(device)

    train(model, dataloaders['train'], dataloaders['valid'], device, params)

    predicted_test_labels, gt_labels = test(model, dataloaders['test'], device, params)

    if predicted_test_labels is None or gt_labels is None:
        return 0, 0, 0

    stop = timeit.default_timer()
    run_time = stop - start

    # np.savetxt(filename, np.asarray(predicted_test_labels))

    correct = 0
    total = 0
    for label, prediction in zip(gt_labels, predicted_test_labels):
        total += label.size(0)
        correct += (prediction.cpu().numpy() == label.cpu().numpy()).sum().item()  # assuming your model runs on GPU

    accuracy = float(correct) / total

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    return correct, accuracy, run_time


"""Main loop. Run time and total score will be shown below."""


def run_on_dataset(dataset_name, device):
    max_run_time = 200
    if dataset_name == "MNIST":
        min_thres = 0.82
        max_thres = 0.92

    elif dataset_name == "CIFAR10":
        min_thres = 0.28
        max_thres = 0.38

    correct_predict, accuracy, run_time = run(dataset_name, device)

    score = compute_score(accuracy, run_time, min_thres, max_thres, max_run_time)
    result = OrderedDict(correct_predict=correct_predict,
                         accuracy=accuracy, score=score,
                         run_time=run_time)
    return result, score


def main():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print('Running on GPU: {}'.format(torch.cuda.get_device_name(0)))
    else:
        device = torch.device("cpu")
        print('Running on CPU')

    result_all = OrderedDict()
    score_weights = [0.5, 0.5]
    scores = []
    for dataset_name in ["MNIST", "CIFAR10"]:
        result_all[dataset_name], this_score = run_on_dataset(dataset_name, device)
        scores.append(this_score)
    total_score = [score * weight for score, weight in zip(scores, score_weights)]
    total_score = np.asarray(total_score).sum().item()
    result_all['total_score'] = total_score
    with open('result.txt', 'w') as f:
        f.writelines(pformat(result_all, indent=4))
    print("\nResult:\n", pformat(result_all, indent=4))


if __name__ == '__main__':
    main()


Running on GPU: Tesla T4
Accuracy of the network on the 10000 test images: 92 %
64.47983597500024
Files already downloaded and verified
Files already downloaded and verified
Accuracy of the network on the 10000 test images: 38 %
66.65423969199992

Result:
 OrderedDict([   (   'MNIST',
                    OrderedDict([   ('correct_predict', 9210),
                                    ('accuracy', 0.921),
                                    ('score', 100.0),
                                    ('run_time', 64.47983597500024)])),
                (   'CIFAR10',
                    OrderedDict([   ('correct_predict', 3836),
                                    ('accuracy', 0.3836),
                                    ('score', 100.0),
                                    ('run_time', 66.65423969199992)])),
                ('total_score', 100.0)])


Part2

In [ ]:
import torch
import timeit
from pprint import pformat

#from A2_submission import get_dataset, tune_hyper_parameter

torch.multiprocessing.set_sharing_strategy('file_system')


def main():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print('Running on GPU: {}'.format(torch.cuda.get_device_name(0)))
    else:
        device = torch.device("cpu")
        print('Running on CPU')

    dataloaders, params = get_dataset('CIFAR10')

    start = timeit.default_timer()

    tune_hyper_parameter(dataloaders, device, params)

    stop = timeit.default_timer()
    run_time = stop - start

    print("\nrun_time:\n", pformat(run_time))


if __name__ == '__main__':
    main()


Running on GPU: Tesla T4
Files already downloaded and verified
Files already downloaded and verified

Optimal performance: Validation Accuracy: 0.391, with SGD optimizer using hyper parameters:
{'accuracy': 0.39088, 'learning_rate': 0.01, 'momentum': 0.9} 

Optimal regularization hyper parameters:
{'value': 0} 

run_time:
 1189.9560726139998
